In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [13]:
from proj1_helpers import *
from costs import *
DATA_TRAIN_PATH = '../train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

Additional loss and helper functions

In [6]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return np.exp(t)/(1+np.exp(t))

def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly=np.ones([x.shape[0],degree+1])
    for i in range(0,x.shape[0]):
        for j in range(1, degree+1):
            poly[i][j]=np.power(x[i],j)
    return poly

def compute_rmse(y,tx,w):
    e=y-(tx @ w)
    return math.sqrt(1/y.shape[0]*(e @ e))


def calculate_loss_logistic_regression(y, tx, w):
    """compute the cost by negative log likelihood."""
    a=0
    for n in range(y.shape[0]):    
        a+=np.log(1+np.exp(tx[n].T @ w))-y[n]*tx[n].T @ w
    return a[0]

def build_tx(x):
    return tx = np.c_[np.ones((y.shape[0], 1)), x]

Gradients calculations

In [16]:
def compute_gradient_MSE(y,tx,w):
    """Compute the gradient."""
    e=y-(tx @ w)
    return -1/y.shape[0]*tx.T @ e

def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient for batch data."""
    return compute_gradient(y,tx,w)

def compute_gradient_MAE(y,tx,w):
    sumX=0
    sumY=0
    for n in range(0,y.shape[0]):
        temp=y[n]-w[0]-w[1]*tx[n,1]
        if(temp>0):
            sumX=sumX-1
            sumY=sumY-tx[n,1]
        if(temp<0):
            sumX=sumX+1
            sumY=sumY+tx[n,1]
    return np.array([sumX/y.shape[0],sumY/y.shape[0]])

def calculate_gradient_logistic_regression(y, tx, w):
    """compute the gradient of loss."""
    return tx.T @ (sigmoid(tx @ w)-y)

def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    # calculate hessian: 
    S=np.zeros((y.shape[0],y.shape[0]))
    for i in range(0,y.shape[0]):
        S[i,i]=sigmoid(tx[i].T @ w)*(1-sigmoid(tx[i].T @ w))
    return tx.T @ S @ tx


def logistic_regression_newton(y, tx, w):
    """return the loss, gradient, and hessian."""
    # return loss, gradient, and hessian:
    return calculate_loss_logistic_regression(y,tx,w),calculate_gradient_logistic_regression(y,tx,w),calculate_hessian(y,tx,w)

def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    # return loss, gradient, and hessian:
    loss,gradient,hessian=logistic_regression_newton(y,tx,w)
    loss+= lambda_*np.sum(w*w)
    gradient+=lambda_*np.sum(2*w)
    hessian+=lambda_*2*w.shape[0]
    return loss,gradient,hessian

Gradient descent (one step)

In [15]:
def learning_by_gradient_descent_logistic_regression(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    # compute the cost:
    loss=calculate_loss_logistic_regression(y,tx,w)
    # compute the gradient:
    gradient=calculate_gradient_logistic_regression(y,tx,w)
    # update w:
    w=w-gamma*gradient
    return loss, w

def learning_by_newton_method(y, tx, w, gamma):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    # return loss, gradient and hessian:
    loss,gradient,hessian = logistic_regression_newton(y,tx,w)
    # update w:
    w=w-gamma * np.linalg.inv(hessian) @ gradient
    return loss, w

def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    # return loss, gradient and hessian:
    
    loss,gradient,hessian = penalized_logistic_regression(y,tx,w,lambda_)
    # update w:
    w=w-gamma * np.linalg.inv(hessian) @ gradient
    return loss, w


# DO GD linear regression with mse
def learning_by_GD_mse(y,tx,w,lambda_):
    return compute_loss(y,tx,w),w-lambda_*compute_gradient_MSE(y,tx,w)
# DO SGD linear regression with mse
def learning_by_SGD_mse(y,tx,w,lambda_):
    return compute_loss(y,tx,w),w-lambda_*compute_stoch_gradient(y,tx,w)

Normal equations

In [10]:
def least_squares(y, tx):
    """calculate the least squares solution."""
    w=np.linalg.inv(tx.T @ tx) @ tx.T @ y
    e= y - (tx @ w)
    mse=1/(2*y.shape[0])*(e.T @ e)
    return w,mse

def ridge_regression(y, tx, lamb):
    """implement ridge regression."""
    return np.linalg.solve((tx.T @ tx)+lamb*np.identity(tx.shape[1]), tx.T @ y)


ML Functions

In [18]:
def gradient_descent_mse(y, tx, initial_w, max_iters, gamma): 
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss,w=learning_by_GD_mse(y,tx,w,gamma)
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
    return losses, ws

def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iter, gamma):
    """Stochastic gradient descent algorithm."""
    ws = [initial_w]
    losses = []
    w = initial_w
    minibatchs = batch_iter(y, tx, batch_size, num_batches=math.floor(y.shape[0]/batch_size))
    for n_iter in range(0,np.min([max_iter,num_batches])):
        # compute gradient and loss
        minibatch=minibatchs.__next__()
        loss,w=learning_by_SGD_mse(minibatch[0],minibatch[1],w)
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
    return losses, ws

def logistic_regression_gradient_descent(y, x, max_iters, threshold,gamma):
    # init parameters
    losses = []
    tx=build_tx(x)
    w = np.zeros((tx.shape[1], 1))
    ws=[]
    ws.append(w)
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent_logistic_regression(y, tx, w, gamma)
        # log info
        if iter % 1000 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        ws.append(w)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            print("The loss={l}".format(l=calculate_loss(y, tx, w)))
            break
    return losses,ws

def logistic_regression_newton_method(y, x,max_iters,threshold,gamma):
    # init parameters
    losses = []
    tx=build_tx(x)
    w = np.zeros((tx.shape[1], 1))
    ws=[]
    ws.append(w)
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, gamma)
        # log info
        if iter % 500 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        ws.append(w)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_newton_method")
    print("The loss={l}".format(l=calculate_loss(y, tx, w)))

def logistic_regression_penalized_gradient_descent(y, x,max_iters,threshold,gamma,lambda_):
    # init parameters
    losses = []
    tx=build_tx(x)
    w = np.zeros((tx.shape[1], 1))
    ws=[]
    ws.append(w)
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 500 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        ws.append(w)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_penalized_gradient_descent")
    print("The loss={l}".format(l=calculate_loss(y, tx, w)))
    
    
def ridge_regression(x, y, degree,lambda_, ratio, seed):
    """ridge regression demo."""
    # form train and test data with polynomial basis function: TODO
    # ***************************************************
    train_tx=build_poly(x,degree)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression with different lambda: TODO
    # ***************************************************
    weight = ridge_regression(y,train_tx,lambda_)
    loss=compute_RMSE(train_y,train_tx,weight)
    return loss,weight
    
    


## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '../test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)